<a href="https://colab.research.google.com/github/elsonyeh/computer-programming-and-application/blob/main/B117610020_%E4%BD%BF%E7%94%A8%E7%B6%B2%E9%A0%81%E7%88%AC%E8%9F%B2%E6%8A%80%E8%A1%93%E6%93%B7%E5%8F%96_104_%E7%A7%91%E6%8A%80%E8%81%B7%E7%BC%BA%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## 題目：使用網頁爬蟲技術擷取 104 科技職缺資料

## 摘要

本報告旨在透過網路爬蟲技術，透過 104 前端職缺搜尋 API 進行科技職缺資料擷取，穩定擷取多個職缺關鍵字的完整職缺資訊。透過 requests 模組搭配 pandas 進行資料整理，擷取包含職缺名稱、公司名稱、地點、薪資、職缺連結等欄位，並儲存為 CSV 格式，可作為後續職缺分析之基礎資料。

## 引言

科技業持續快速發展，科技人才市場競爭激烈。透過自動化爬蟲技術擷取大量職缺資料，有助於掌握產業趨勢、技能需求與公司需求。本專案以 104 為主要資料來源，實作完整爬蟲流程，並擷取多個熱門科技職位之職缺資訊。

## 方法

- 目標網站描述
  - 目標網站：104人力銀行（https://www.104.com.tw/）
  - 目標欄位：職缺名稱、公司名稱、薪資、地點、職缺說明、需求條件與公司福利
  - 資料來源：
    - 搜尋API：https://www.104.com.tw/jobs/search/list
- 工具與技術
  - 使用 Python 的 `requests` 套件發送 HTTP 請求並直接存取 104 內部 API
  - 使用 `pandas` 套件進行資料整理與儲存
  - 使用 Google Chrome 開發者工具協助觀察 API 請求參數與結構


#### 匯入套件

In [221]:
import requests
import pandas as pd
import time

from IPython.display import display
# 顯示優化設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

#### 設定搜尋關鍵字列表與資料筆數

In [239]:
# 要搜尋的多個職缺關鍵字
keywords = [
    "軟體工程師", "資料工程師", "資料科學家", "人工智慧", "機器學習", "大數據", "雲端工程師", "資安工程師", "DevOps",
    "前端工程師", "後端工程師", "全端工程師", "系統工程師",
    "網路工程師", "資料庫工程師"
]

# 每個關鍵字要擷取幾筆
target_count = 30

#### 取得職缺列表

In [240]:
def get_job_list(keyword, page):
    url = "https://www.104.com.tw/jobs/search/list"
    params = {
        "ro": "1",
        "keyword": keyword,
        "order": "11",
        "asc": "0",
        "page": page,
        "mode": "l"
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.104.com.tw/"
    }

    res = requests.get(url, headers=headers, params=params)

    if res.status_code != 200:
        print(f"API擷取失敗，狀態碼：{res.status_code}")
        return []

    data = res.json()
    job_list = data.get("data", {}).get("list", [])

    jobs = []
    for job in job_list:
        jobs.append({
            "職缺名稱": job.get("jobName", ""),
            "公司名稱": job.get("custName", ""),
            "工作地點": job.get("jobAddrNoDesc", ""),
            "薪資": job.get("salaryDesc", ""),
            "職缺連結": f"https://www.104.com.tw/job/{job.get('jobNo')}",
        })

    return jobs

#### 主程式

In [243]:
#job_type=查詢內容/target_success_count=資料計數
def crawl_jobs_multi(keywords, target_count):
    all_data = []

    for keyword in keywords:
        print(f"\n正在擷取關鍵字：{keyword}")
        page = 1
        keyword_data = []

        while len(keyword_data) < target_count:
            jobs = get_job_list(keyword, page)
            if not jobs:
                break

            for job in jobs:
                keyword_data.append(job)
                if len(keyword_data) >= target_count:
                    break

            page += 1
            time.sleep(0.5)

        print(f"{keyword} 擷取完成，共 {len(keyword_data)} 筆")
        all_data.extend(keyword_data[:target_count])

    return pd.DataFrame(all_data)

#### 資料整理與儲存

In [244]:
df = crawl_jobs_multi(keywords, target_count)
df.to_csv("104職缺資料.csv", index=False)
print(f"\n全部關鍵字擷取完成！共收集 {len(df)} 筆資料。")

display(pd.concat([df.head(5), df.tail(5)]))


正在擷取關鍵字：軟體工程師
軟體工程師 擷取完成，共 30 筆

正在擷取關鍵字：資料工程師
資料工程師 擷取完成，共 30 筆

正在擷取關鍵字：資料科學家
資料科學家 擷取完成，共 30 筆

正在擷取關鍵字：人工智慧
人工智慧 擷取完成，共 30 筆

正在擷取關鍵字：機器學習
機器學習 擷取完成，共 30 筆

正在擷取關鍵字：大數據
大數據 擷取完成，共 30 筆

正在擷取關鍵字：雲端工程師
雲端工程師 擷取完成，共 30 筆

正在擷取關鍵字：資安工程師
資安工程師 擷取完成，共 30 筆

正在擷取關鍵字：DevOps
DevOps 擷取完成，共 30 筆

正在擷取關鍵字：前端工程師
前端工程師 擷取完成，共 30 筆

正在擷取關鍵字：後端工程師
後端工程師 擷取完成，共 30 筆

正在擷取關鍵字：全端工程師
全端工程師 擷取完成，共 30 筆

正在擷取關鍵字：系統工程師
系統工程師 擷取完成，共 30 筆

正在擷取關鍵字：網路工程師
網路工程師 擷取完成，共 30 筆

正在擷取關鍵字：資料庫工程師
資料庫工程師 擷取完成，共 30 筆

全部關鍵字擷取完成！共收集 450 筆資料。


,職缺名稱,公司名稱,工作地點,薪資,職缺連結
0,軟體工程師 Golang,線上探索科技股份有限公司,台北市大同區,待遇面議,https://www.104.com.tw/job/14271913
1,"【北士科】Software Engineer, Legal Automation &amp;...",沛思坦網路股份有限公司,台北市北投區,待遇面議,https://www.104.com.tw/job/14559109
2,軟體工程師,衡碁科技股份有限公司,新竹縣湖口鄉,"月薪42,000~60,000元",https://www.104.com.tw/job/11446450
3,程式軟體工程師,泳鉅鑫再生塑料股份有限公司,台南市永康區,"月薪50,000~80,000元",https://www.104.com.tw/job/14669353
4,"軟體工程師（software engineer)（Tijuana, Mexico)",松澤企業股份有限公司,墨西哥,"年薪1,000,000~2,000,000元",https://www.104.com.tw/job/14659513
445,PHP 工程師（原生PHP + MySQL開發｜直銷專案｜具備前端基礎）,立馬吃一鍋_立馬雲集股份有限公司,台北市大安區,"月薪38,000~50,000元",https://www.104.com.tw/job/14372299
446,資訊軟體工程師,侑盈科技有限公司,新北市三重區,"月薪35,000元以上",https://www.104.com.tw/job/11257600
447,GIS地理資訊系統工程師,河畔果管理顧問有限公司,台北市信義區,待遇面議,https://www.104.com.tw/job/13616913
448,AI研發工程師,樂倍達數位科技股份有限公司,台北市中山區,待遇面議,https://www.104.com.tw/job/14657328
449,資料庫管理人員,台灣基康股份有限公司,台北市信義區,"月薪33,000~35,000元",https://www.104.com.tw/job/11875621


## 結果


- 成功透過 104 API 擷取 15 組科技職缺關鍵字，每組擷取 30 筆，共取得 450 筆完整職缺資訊並整理儲存。

- 數據存儲方式
  - 將數據存儲為 CSV 格式，方便後續分析。

## 問題與挑戰

- 104 網站職缺列表改為 JavaScript 動態載入，必須改採內部 API 擷取
- API 回傳資料為 JSON 格式，需正確解析多層巢狀結構
- 每次請求需注意適當延遲以避免被封鎖
- 需控制請求速率避免觸發反爬蟲機制
- 詳細頁資料仍需額外 API 取得與整合

## 結論


- 本專案成功實作完整 104 API 科技職缺自動化擷取流程，透過 104 搜尋 API 快速穩定地擷取多關鍵字完整職缺資訊，可用於後續職缺分析與科技人才市場趨勢研究。

## 參考文獻

* [104人力銀行](https://www.104.com.tw/)